# Исследование надёжности заёмщиков

# Содержание

- [Описание проекта](#1)
- [Общая информация о данных](#2)
- [Предобработка данных](#3)
- [Исследование надёжности заёмщиков](#4)
- [Заключение](#5)

# Описание проекта <a id='1'></a>

**Цель: выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.**

Заказчик — кредитный отдел банка. 
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Общая информация о данных <a id='2'></a>

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.info()
display(df[df['total_income'].isna()].head(10))
print(df.isna().mean()*100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64


- В двух столбцах (days_employed, total_income) есть пропущенные значения, в каждом столбце примерно 10% от общего масива. Возможными  причинами появления пропусков могут быть: нежелание госсужащих (сотрудников, компаньонов) афишировать свои персональные данные, отсутствие стабильного дохода у пенсионеров. Однако исключать данные строки из общего массива данных не самое лучшее решение, ввиду наличия в них ценных сведений по интерисующим заказчика критериям. С учетом того что это количественные данные - лучше заменить пропуски медианным значением (среднее может сильно сместить данные).
- Целесообразно заменить вещественный тип данных в столбце total_income на целочисленный.

In [2]:
display(df.tail(2))
display(df.head(2))
print()
print('education:', df['education'].unique())
print()
print('children:', df['children'].unique())
print()
print('education_id:', df['education_id'].unique())
print()
print('family_status:', df['family_status'].unique())
print()
print('income_type:', df['income_type'].unique())
print()
print('purpose:', df['purpose'].unique())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля



education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

children: [ 1  0  3  2 -1  4 20  5]

education_id: [0 1 2 3 4]

family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

purpose: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строит

* В данных выявлены аномалии:  
отрицательное количество дней трудового стажа и очень большие заначения стажа (свыше 1000 лет) в столбце days_employed (объяснить появление таких данных можно ошибкой при выгрузке данных; использовать этот столбец для анализа затруднительно); 
орицатетельное и очень большое количество детей в столбце children (можно объястить опечаткой при вводе данных; этим данным следут уделить дополнительное внимание).
- Необходимо заменить неявные дубликаты: например в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв.
- Значения в столбце purpose следует распределить по укрупнённым категориям.

# Предобработка данных <a id='3'></a>

Заменим пропуски в столбцах days_employed, total_income медианным значением.

In [3]:
med_days = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(value = med_days)

med_total = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(value = med_total)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Обработка грубых ошибок. Заменим, вероятно появившиеся в резутальтате некорректного ввода или выгрузки, значения количества детей 20 и -1 на 2 и 1 соответственно.

In [4]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1
print('children:', df['children'].unique()) #проверка


children: [1 0 3 2 4 5]


Заменим вещественный тип данных в столбце total_income на целочисленный.

In [5]:
df['total_income'] = df['total_income'].astype('int') 

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Обработаем неявные дубликаты (разный регистр в написании значений), которые могли появиться в результате ввода данных разными сотрудниками банка. Приведем к "змеиному" стилю значения столбцов education и family_status.

In [7]:
df['education'] = df['education'].str.lower() 
df['family_status'] = df['family_status'].str.lower()
print('education:', df['education'].unique()) #проверка
print('family_status:', df['family_status'].unique()) #проверка

education: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


Проверим наличие явных дубликатов

In [8]:
print ('Дубликатов в таблице:', df.duplicated().sum())

Дубликатов в таблице: 71


Для удаления явных дубликатов применим метод позволяющий как удалить дубликаты, так и заменить индексацию на новую.

In [9]:
df = df.drop_duplicates().reset_index(drop=True) 
print ('Дубликатов в таблице:', df.duplicated().sum()) #проверка

Дубликатов в таблице: 0


Создим два новых датафрейма со столбцами:
education_id и education — в первом;
family_status_id и family_status — во втором.

In [10]:
education_dict = df[['education_id', 'education']]
family_status_dict = df[['family_status_id', 'family_status']]

In [11]:
df.drop(columns = ['education', 'family_status'], axis = 1)
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

На основании диапазонов, указанных ниже, создадим функцию для формирования категорий:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.

In [12]:
def income_group(income): 
    if income >= 0 and income <=30000: 
        return 'E'
    if income >= 30001 and income <=50000: 
        return 'D'
    if income >= 50001 and income <=200000: 
        return 'C'
    if income >= 200001 and income <=1000000: 
        return 'B'
    if income >= 1000001: 
        return 'A'

Создадим столбец total_income_category с категориями: 0–30000 — 'E'; 30001–50000 — 'D'; 50001–200000 — 'C'; 200001–1000000 — 'B'; 1000001 и выше — 'A'.

In [13]:
df['total_income_category'] = df['total_income'].apply(income_group) 


Создадим функцию для преобразования данных столбца purpose и сформируем новый столбец purpose_category, в который войдут следующие категории:'операции с автомобилем',
'операции с недвижимостью', 'проведение свадьбы', 'получение образования'.

In [14]:
def purpose_group(purpose):
    a = 'жиль'
    b = 'недв'
    c = 'авто'
    d = 'образ'
    e = 'свадь'
    if a in purpose: 
        return 'операции с недвижимостью'
    if b in purpose: 
        return 'операции с недвижимостью'
    if c in purpose: 
        return 'операции с автомобилем'
    if d in purpose: 
        return 'проведение свадьбы'
    if e in purpose: 
        return 'получение образования'
        
    
df['purpose_category'] = df['purpose'].apply(purpose_group)

print(df['purpose_category'].value_counts()) #проверка


операции с недвижимостью    10811
операции с автомобилем       4306
проведение свадьбы           4013
получение образования        2324
Name: purpose_category, dtype: int64


# Выводы

В результате предварительной обработки данных:
- Пропущенные зачения в столбцах days_employed, total_income заменены медианным значением.
- Вещественный тип данных в столбце total_income заменён на целочисленный.
- Орицатетельное количество детей в столбце children (-1) замено на 1. 
- Анамально большое малколичество детей в столбце children (20) заменено на 2.
- Исключены явные дубликаты.
- Заменены неявные дубликаты в столбце education.
- Значения в столбцах purpose  и total_income распределены по укрупнённым категориям.

# Исследование надёжности заёмщиков <a id='4'></a>

## Анализ зависимости между количеством детей и возвратом кредита в срок. 

Определим есть ли зависимость между количеством детей и возвратом кредита в срок. Для анализа используем сводные таблицы. Расчитаем количество просроченных кредитов.

In [15]:

data_pivot_cat_sum = df.pivot_table(index=['children'], values='debt', aggfunc='sum') 
display(data_pivot_cat_sum.head(20))


,debt
children,
0,1063
1,445
2,202
3,27
4,4
5,0


На первый взгляд, чем больше детей тем лучше платежеспособность и меньше просроченных кредитов. Однако если найти соотношение просроченных кредитов к общему количеству выданных в каждой категории увидим обратную зависимость. 

In [16]:

data_pivot_cat_sum = df.pivot_table(index=['children'], values='debt', aggfunc='sum') 
data_pivot_cat_count = df.pivot_table(index=['children'], values='debt', aggfunc='count') 
data_pivot_cat = data_pivot_cat_sum / data_pivot_cat_count * 100
display(data_pivot_cat.head(20))


,debt
children,
0,7.543822
1,9.165808
2,9.492481
3,8.181818
4,9.756098
5,0.000000


- Как видно из таблицы, **доля невозвращенных кредитов растёт с увеличением количества детей**. Снижение доли невозвращенных кредитов для 3-х детей можно объяснить наличием льгот для многодетных семей. Нулевые значения указывают на то что, кредиты в данной категории не выдавались.
- Схожая тенденция наблюдается для всех кредитных категорий, за исключением категории "получение образования". 

In [17]:

data_pivot_cat_sum = df.pivot_table(index=['children'], columns='purpose_category', values='debt', aggfunc='sum') 
data_pivot_cat_count = df.pivot_table(index=['children'], columns='purpose_category', values='debt', aggfunc='count') 
data_pivot_cat = data_pivot_cat_sum / data_pivot_cat_count * 100
display(data_pivot_cat.head(20))


purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
children,,,,
0,8.541301,6.728866,7.516340,8.667676
1,10.677618,8.093889,9.550562,10.273973
2,12.048193,8.403361,6.696429,11.244019
3,8.333333,7.692308,15.625000,5.797101
4,10.000000,14.285714,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000


## Анализ зависимости между семейным положением и возвратом кредита в срок. 

In [18]:
data_pivot_fam_sum = df.pivot_table(index=['family_status'], values='debt', aggfunc='sum') 
data_pivot_fam_count = df.pivot_table(index=['family_status'], values='debt', aggfunc='count') 
data_pivot_fam = data_pivot_fam_sum / data_pivot_fam_count * 100
display(data_pivot_fam)


,debt
family_status,
в разводе,7.112971
вдовец / вдова,6.569343
гражданский брак,9.347145
женат / замужем,7.545182
не женат / не замужем,9.750890


Из данных таблицы видно, что наибольшая доля невозвращенных в срок редитов 9,75% и 9,35% приходится соответственно на категории "не женат / не замужем" и "гражданский брак". Наименьшая доля невозвращенных в срок редитов 7,55%, 7,11% и 6,57% приходится соответственно на категории "женат / замужем", "в разводе" и "вдовец / вдова".

Проанализируем, измененение отмеченной зависимости в разрезе кредитных категорий.

In [19]:
data_pivot_fam_sum = df.pivot_table(index=['family_status'], columns='purpose_category', values='debt', aggfunc='sum') 
data_pivot_fam_count = df.pivot_table(index=['family_status'], columns='purpose_category', values='debt', aggfunc='count') 
data_pivot_fam = data_pivot_fam_sum / data_pivot_fam_count * 100
display(data_pivot_fam)


purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
family_status,,,,
в разводе,7.473310,6.952663,NaN,7.142857
вдовец / вдова,9.174312,5.166052,NaN,7.537688
гражданский брак,11.751152,9.201213,8.003442,14.851485
женат / замужем,8.369883,6.934932,NaN,8.323699
не женат / не замужем,12.872841,8.145363,NaN,10.745234


Как видно из данных таблицы, отмеченная выше тенденция справедлива для всех кредитных каегорий:  наибольшая доля невозвращенных в срок редитов приходится на категории "не женат / не замужем" и "гражданский брак", наименьшая доля невозвращенных в срок редитов приходится на категории "женат / замужем", "в разводе" и "вдовец / вдова". Отсутствие значений (NaN) в столбце "получение образования" указывает на то что, кредиты в данной категории не выдавались.

## Анализ зависимости между уровнем дохода и возвратом кредита в срок. 

In [20]:
data_pivot_inc_sum = df.pivot_table(index=['total_income_category'], values='debt', aggfunc='sum') 
data_pivot_inc_count = df.pivot_table(index=['total_income_category'], values='debt', aggfunc='count') 
data_pivot_inc = data_pivot_inc_sum / data_pivot_inc_count * 100
display(data_pivot_inc.head(20))


,debt
total_income_category,
A,8.000000
B,7.062091
C,8.491508
D,6.000000
E,9.090909


Очевидной зависимости между уровнем дохода и возвратом кредита в срок не наблюдается. Проанализируем как влияет уровень дохода на погашение кредита в срок в разрезе кредитных категорий.

In [21]:
data_pivot_inc_sum = df.pivot_table(index=['total_income_category'], columns='purpose_category', values='debt', aggfunc='sum') 
data_pivot_inc_count = df.pivot_table(index=['total_income_category'], columns='purpose_category', values='debt', aggfunc='count') 
data_pivot_inc = data_pivot_inc_sum / data_pivot_inc_count * 100
display(data_pivot_inc.head(20))


purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
total_income_category,,,,
A,0.000000,5.882353,0.000000,25.000000
B,8.341416,6.697819,5.137615,7.803790
C,9.596749,7.468622,8.913168,9.798746
D,13.888889,3.571429,8.823529,2.631579
E,0.000000,18.181818,0.000000,0.000000


Анализируя долю возврата кредита в срок от уровня дохода в разрезе категорий, однозначного вывода о наличии зависимости о между уровнем дохода и возвратом кредита в срок сделать нельзя. Проанализируем объём выборок в разрезе уровня доходов и платёжных категориий.

In [22]:
display(data_pivot_inc_sum.head(20))
display(data_pivot_inc_count.head(20))

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
total_income_category,,,,
A,0,1,0,1
B,86,172,28,70
C,307,601,155,297
D,10,6,3,2
E,0,2,0,0


purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
total_income_category,,,,
A,2,17,2,4
B,1031,2568,545,897
C,3199,8047,1739,3031
D,72,168,34,76
E,2,11,4,5


Исключив из анализа, категории дохода A, D, E, объём выборок в которых пренебрежимо мал (менее 3%) по сравнению с выборками В и С. Можно сделать вывод о наличии зависимости о между уровнем дохода и возвратом кредита в срок - **доля кредитов невозвращённых в срок уменьшается с увеличением уровня дохода для всех кредитных категорий.**  

## Оценка влияния цели кредита на его возврат в срок. 

In [23]:
data_pivot_pur_sum = df.pivot_table(index=['purpose_category'], values='debt', aggfunc='sum') 
data_pivot_pur_count = df.pivot_table(index=['purpose_category'], values='debt', aggfunc='count') 
data_pivot_pur = data_pivot_pur_sum / data_pivot_pur_count * 100
display(data_pivot_pur)

,debt
purpose_category,
операции с автомобилем,9.359034
операции с недвижимостью,7.233373
получение образования,8.003442
проведение свадьбы,9.220035


Наименьшая доля невозвращенных в срок кредитов принадлежит категории "операции с недвижимостью" - 7,23%, наибольшая - "операции с автомобилем" - 9,36%. Проанализируем, каким категориям кредитополучателей свойственно задерживание платежей.

In [24]:
data_pivot_pur_sum = df.pivot_table(index=['purpose_category'], columns='family_status',  values='debt', aggfunc='sum') 
data_pivot_pur_count = df.pivot_table(index=['purpose_category'], columns='family_status',  values='debt', aggfunc='count') 
data_pivot_pur = data_pivot_pur_sum / data_pivot_pur_count * 100
display(data_pivot_pur)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
purpose_category,,,,,
операции с автомобилем,7.473310,9.174312,11.751152,8.369883,12.872841
операции с недвижимостью,6.952663,5.166052,9.201213,6.934932,8.145363
получение образования,NaN,NaN,8.003442,NaN,NaN
проведение свадьбы,7.142857,7.537688,14.851485,8.323699,10.745234


Из данных таблицы видно, что существенно увеличивают долю просроченных платежей такие категории как: "гражданский брак" и "не женат / не замужем". 

**Выводы**

В результате анализа данных о платёжеспособности клиентов:

- найдена зависимость между количеством детей и возвратом кредита в срок.
- выделены категории граждан по семейному положению имеющие большую долю задержек платежей по кредитам.
- найдена зависимость между уровнем дохода и возвратом кредита в срок.
- определены цели кредита по которым с большей и меньшей долей задержек платежей по кредитам.


# Заключение <a id='5'></a>

По результатам исследования:
1. установлено, что **доля невозвращенных кредитов растёт с увеличением количества детей**;
отмечено, снижение доли невозвращенных кредитов для кредитополучателей с 3-мя детьми, что объясняется наличием льгот для многодетных семей;
2. определено, что **наибольшая доля невозвращенных в срок редитов 9,75% и 9,35% приходится соответственно на категории "не женат / не замужем" и "гражданский брак", наименьшая доля невозвращенных в срок кредитов 7,55%, 7,11% и 6,57% приходится соответственно на категории "женат / замужем", "в разводе" и "вдовец / вдова"**;
3. установлено, что **доля кредитов невозвращённых в срок уменьшается с увеличением уровня дохода для всех кредитных вкатегорий;**
4. определено, что **наименьшая доля невозвращенных в срок кредитов принадлежит категории "операции с недвижимостью" - 7,23%, наибольшая - "операции с автомобилем" - 9,36%**, при этом существенно увеличивают долю просроченных платежей такие категории как: "гражданский брак" и "не женат / не замужем".

